In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib.ticker import MaxNLocator

In [2]:
# load data 
df = pd.read_csv('../Data/example_data.csv', delimiter=',')
df.head()

,X1,X2,X3,X4,X5,X6,y
0,0.496714,-0.678495,0.348286,-1.980572,high,medium,0
1,-0.138264,-0.305499,0.283324,-1.054986,low,low,0
2,0.647689,-0.597381,-0.936520,-0.587028,high,high,0
3,3.523030,0.110418,0.579584,0.149669,low,low,1
4,-0.234153,1.197179,-1.490083,1.024162,low,high,0


### 5.4.1 Geschiktheid en effectiviteit potentiële profileringskenmerken 

#### Kwantitatieve toetsingsmethoden

##### **Gebaseerd op inclusie**

**Voorbeeld 1:**

Toetsen van vermoeden dat kenmerk $X_1$ verband houdt met doelvariabele $y$:
- $H_0: p(Y=1|X_1>0) = p(Y=1|X_1≤0)$ (kans op positieve uitkomst doelvariabele voor X1>0 is gelijk aan de kans op positieve uitkomst doelvariabele voor X1≤0)
- $H_1: p(Y=1|X_1>0) > p(Y=1|X_1≤0)$ (kans op positieve uitkomst doelvariabele voor X1>0 is groter dan de kans op positieve uitkomst doelvariabele voor X1≤0)

In [7]:
from statsmodels.stats.proportion import proportions_ztest

# Create boolean masks for X1 > 0 and X1 <= 0
mask_X1_gt_0 = df['X1'] > 0
mask_X1_le_0 = df['X1'] <= 0

# Count successes (y=1) and total for each group
successes = [df.loc[mask_X1_gt_0, 'y'].sum(), df.loc[mask_X1_le_0, 'y'].sum()]
nobs = [mask_X1_gt_0.sum(), mask_X1_le_0.sum()]

# Perform one-sided z-test for proportions (alternative: larger)
stat, pval = proportions_ztest(successes, nobs, alternative='larger')

print(f"Z-statistiek: {stat:.4f}")
print(f"p-waarde: {pval:.4f}")
if pval < 0.05:
    print("Nulhypothese verworpen: kans op positieve uitkomst doelvariabele (Y=1) is significant hoger voor X1>0.")
else:
    print("Nulhypothese niet verworpen: geen statistisch significant verschil gevonden.")

Z-statistiek: 28.1060
p-waarde: 0.0000
Nulhypothese verworpen: kans op positieve uitkomst doelvariabele (Y=1) is significant hoger voor X1>0.


##### **Profiel met j≤k kenmerken**

**Voorbeeld 2:**

Toetsen van vermoeden dat profiel $X_1>0, X_2>0, X_5=\text{laag}$ verband houdt met doelvariabele $y$:
- $H_0: p(Y=1|X_1>0, X_2>0, X_5=\text{laag}) = p(Y=1|X_1≤0, X_2≤0, X_5=\text{laag})$ (kans op positieve uitkomst doelvariabele voor $X_1>0, X_2>0, X_5=\text{laag}$ is gelijk aan de kans op positieve uitkomst doelvariabele voor $X_1≤0, X_2≤0, X_5\neq\text{laag}$)
- $H_1: p(Y=1|X_1>0, X_2>0, X_5=\text{laag}) > p(Y=1|X_1≤0, X_2≤0, X_5=\text{laag})$  (kans op positieve uitkomst doelvariabele voor $X_1>0, X_2>0, X_5=\text{laag}$ is groter dan de kans op positieve uitkomst doelvariabele voor $X_1≤0, X_2≤0, X_5\neq\text{laag}$)

ANOVA

In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Convert categorical variable X5 to categorical dtype if not already
df['X5'] = df['X5'].astype('category')

# Fit ANOVA model (ordinary least squares with categorical X5)
model = smf.ols('y ~ X1 + X2 + C(X5)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# Check p-values to assess predictive value
print("\nPredictive value (p < 0.05 indicates significant predictive value):")
print(anova_table['PR(>F)'] < 0.05)

              sum_sq      df            F    PR(>F)
C(X5)       0.101587     2.0     0.786485  0.455471
X1        219.000431     1.0  3390.992997  0.000000
X2          0.001801     1.0     0.027888  0.867377
Residual  645.506879  9995.0          NaN       NaN

Predictive value (p < 0.05 indicates significant predictive value):
C(X5)       False
X1           True
X2          False
Residual    False
Name: PR(>F), dtype: bool


##### **Gebaseerd op exclusie**

**Voorbeeld 3:**

Toetsen van vermoeden dat profiel alle variabelen verband houdt met doelvariabele $y$:
- $H_0$: ...
- $H_1$: ...